In [1]:
#关于文档的一个数据分类
import pandas as pd
import seaborn as sns  #用于绘制热图的工具包
from scipy.cluster import hierarchy  #用于进行层次聚类，话层次聚类图的工具包
from scipy import cluster   
import matplotlib.pyplot as plt
from sklearn import decomposition as skldec #用于主成分分析降维的包
import numpy as np
#import csv
import math
from pandas import Series,DataFrame
from matplotlib.ticker import  MultipleLocator
import networkx as nx

In [2]:
# data=pd.read_excel('NewAllData.xlsx')
data = pd.read_excel('E:/vcdata/vcdata(91-17-new).xlsx')
data = data[data['city']!='--']######将城市为空的数据过滤掉，得到城市风险投资网络图
def getstr(s):
    return str(s)[:4]
data['get_year'] = data['投资时间'].map(getstr)
data = data[data['投资金额(RMB)']!='--']######将城市为空的数据过滤掉，得到城市风险投资网络图
# data = 
### 读取所有机构名称，按照机构名称，画各个网络
# vcJG=data['机构名称'].drop_duplicates().values

In [3]:
################ 弄成矩阵试试 ############

In [5]:
data_2015 = data[data['get_year']=='2015']
group_mech = data_2015.groupby('机构名称')####根据投资机构进行分类
Cityname = data_2015['city'].drop_duplicates().values
matrix_a15 = np.zeros((len(Cityname),len(Cityname)))###初始化矩阵
matrix_a15_2 = np.zeros((len(Cityname),len(Cityname)))###初始化矩阵
for g in group_mech:
    cities = g[1]['city'].values
    amount = g[1]['投资金额(RMB)'].values
    for i in range(len(cities)-1):
        matrix_a15[list(Cityname).index(cities[i])][list(Cityname).index(cities[i+1])] += 1 ###标志边存在
        matrix_a15_2[list(Cityname).index(cities[i])][list(Cityname).index(cities[i+1])] += amount[i] ###标志边存在

In [6]:
data_2000 = data[data['get_year']=='2000']
group_mech20 = data_2000.groupby('机构名称')####根据投资机构进行分类
Cityname20 = data_2000['city'].drop_duplicates().values
matrix_a20 = np.zeros((len(Cityname20),len(Cityname20)))###初始化矩阵
matrix_a20_2 = np.zeros((len(Cityname20),len(Cityname20)))###初始化矩阵
for g in group_mech20:
    cities = g[1]['city'].values
    amount = g[1]['投资金额(RMB)'].values
    for i in range(len(cities)-1):
        matrix_a20[list(Cityname20).index(cities[i])][list(Cityname20).index(cities[i+1])] += 1 ###标志边存在
        matrix_a20_2[list(Cityname20).index(cities[i])][list(Cityname20).index(cities[i+1])] += amount[i] ###标志边存在

In [7]:
def get_edges(matrix_a,city,H):
    index =  []###大城市的标号
    Edges = []###选出点为大城市的边
    for i in range(len(city)):
        if city[i]=='北京市' or city[i]=='上海市' or city[i]=='天津市' or city[i]=='重庆市' or city[i]=='广州市':
            index.append(i)
    edges = list(H.edges)
    for i in index:
        for j in edges:
            if i in j:
                Edges.append(j)
    return(Edges,index)

In [8]:
import community
import networkx as nx
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 8))
G=nx.Graph(matrix_a15)
components = nx.connected_components(G)
largest_component = max(components, key=len)
H = G.subgraph(largest_component)
# compute centrality
centrality = nx.betweenness_centrality(H, k=10, endpoints=True)

# compute community structure
lpc = nx.community.label_propagation_communities(H)
community_index = {n: i for i, com in enumerate(lpc) for n in com}

#### draw graph ####
# pos = nx.spring_layout(H, k=0.15, seed=4572321)
# pos = nx.kamada_kawai_layout(H)

#first compute the best partition
partition = community.best_partition(H)
y =list(partition.values())
#drawing
size = float(len(set(partition.values())))
count = 0.
de = G.degree()
deg = []
de = dict(de)
node_size = [v * 20 for v in list(de.values())]
pos = nx.kamada_kawai_layout(H)  # compute graph layout
Cityname = data_2015['city'].drop_duplicates().values
name=[]
arg = np.argsort(-np.array(list(de.values())))
# print("arg:",arg)
labels = {}
pp = ['BJ','SH','SZ','GZ','TJ','CQ']
for index in range(6):
    labels[arg[index]] = pp[index]

edg = get_edges(matrix_a15,Cityname,H)[0]
# plt.figure(figsize=(8, 8))  # image is 8 x 8 inches
plt.axis('off')
nx.draw_networkx_nodes(H, pos, node_size=node_size,cmap=plt.cm.RdYlBu, node_color=y,alpha=0.8)
nx.draw_networkx_edges(H, pos,edgelist=edg, alpha=0.15)
nx.draw_networkx_labels(H,pos,labels, font_size=12,font_color='black')
plt.savefig('E:/0_论文/0picture/网络_0.png',dpi=500, bbox_inches='tight')
plt.show(H)

ModuleNotFoundError: No module named 'community'